# Running Pygeodyn with Kamodo's CTIPe Model


**GEODYN Run Configuration:**
- ICESat2 with PCE tracking datatype
- Constructed a smaller G2B file to only run over the time we have model data for... this speeds up the time to copy files for quick runs.


**How we added Kamodo :**
- added a third case to the GEODYN options text files that is modified/written by pygeodyn and read by GEODYN at the time of execution.  This `geodyn_option.txt` is read in by the DRAG.f90 subroutine if the `ATMDEN` card is set to `86`.
- If the `kamodo_ctipe` density model is inputted to pygeodyn, DRAG.f90 calls the Kamodo_CTIPe.f90 subroutine which sends  a command line argument to the console to run Kamodo's ctipe wrapper for a single point.  The neutral density and drhodz at that point is then written to a text file and read back in to GEODYN. 
- had to modify `CTIPe_wrapper_fortrancmd.py` to take the Sattime as `YYMMDDHHMMSS`


In [1]:
import copy
import sys
import os.path
import numpy as np

Some things to keep track of:
```
    Date       DOY      Epoch in iisset
    -------------------------------------
    Nov 29     333A     not used
    Nov 30     334    
    Dec 1      335        
    Dec 2      336      2018/12/01 21:00   -- 2018/12/04/ 03:00
    Dec 3      337   
    Dec 4      338    
    Dec 5      339A     not used
```

Figuring out Time stuff:
``` 
  Variable     Value              Desc.
  ----------------------------------------------------------------------
  TGTYMD        410106.0000       GEODYN INTERNAL REFERENCE TIME (YYMMDD)  
  TMGDN1       2430000.5000       GEODYN INTERNAL REFERENCE TIME (JD)
  TMGDN2         30000.0000       GEODYN INTERNAL REFERENCE TIME (MJD)
  REPDIF             0.0000       DIFF IN SECS BTW RUN REF TIME AND DEFAULT REF TIME
  XTMGN              5.0000       NUMBER OF WORDS IN THIS COMMON BLOCK
  
  STARTT             0.0000       start time?
  ESSTRT    2458414869.1840       Run start in integer mjd seconds
  FSSTRT             0.0000       Run start fractional seconds
  45384
  MJDSEC    2458414869             Modified Julian Day Seconds (INT)
 
```
 


In [2]:
import pandas as pd

# year = 2018
# DOY = 335
# secs_uts = 75669.183999997767

# print('day', DOY)
# print(secs_uts/3600, 'hours')


# date_geodyn_str = str(IYMD) + str(IHMS)
# date_geodyn_dt = pd.to_datetime( date_geodyn_str, format='%y%m%d%H%M%S')

# print(date_geodyn_dt)

# epoch_start_dt = pd.to_datetime( , format='%y%m%d%H%M%S')
# add_hours_dt = pd.Series(pd.to_timedelta(9,'h'))
# drag_date_1 = (epoch_start_dt+add_hours_dt).dt.strftime(  '%y%m%d%H%M%S').values[0]

# import time
# import datetime

# unixtime = time.mktime(date_geodyn_dt.timetuple())
# print(unixtime)

In [3]:
### Identify which arcs you want to run:


sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')

#------ A dictionary containing the run parameters ------  
run_params = {}                                  
run_params['arc']              =   ['2018.336']  
run_params['satellite']        =  'icesat2'     
run_params['SpecialRun_name']  =  'TEST_KAMODO'  
run_params['verbose']          =  False          
run_params['action']           =  'run'





In [4]:
%load_ext autoreload
%autoreload 2

from PYGEODYN import Pygeodyn


##### Use copy.deepcopy to copy all levels of dictionary and 
###       allow modification of new variable
run_params3 = copy.deepcopy(run_params)
run_params3['den_model'] =  'msis2'  

### Run pyeodyn for the arcs in the above set.
Obj_Geodyn = Pygeodyn(run_params3)
Obj_Geodyn.RUN_GEODYN()


                      ......... RUNNING GEODYN
Run # 1     Current Time =      18:17:40
Run # 1
Run # 1     Cleaning iisset... :    /data/data_geodyn/inputs/icesat2/setups/iisset.2018.336.bz2
Epoch Start:  2018-12-01 21:00:00
+ ——————————————————————————————————————————————————————————————————————————————————————————————————————
|
| ---------------------- Some run information ----------------------
|
|  Run # 1     IISSET Cleaned      tmp/.../cleaned_setup_2018336
|  Run # 1     Density Model:      msis2
|  Run # 1     GEODYN Version:     Kamodo_pygeodyn_MODS
|  Run # 1     ARC run:            icesat2_2018336_54hr.msis2
|  Run # 1     Output directory:   /data/data_geodyn/results/icesat2/msis2/msis2_acceloffTEST_KAMODO
|  Run # 1     EXAT File:          /data/data_geodyn/inputs/icesat2/external_attitude/EXAT01.2018.336.gz
|
+ ——————————————————————————————————————————————————————————————————————————————————————————————————————

Run # 1          Running IIS
Run # 1          No errors in